## Linear Algebra

Attempt at streamlining the whole code, as is quite messy now.

### Imports

In [6]:
import apogee.tools.read as apread
import apogee.tools.path as apogee_path
from apogee.tools import bitmask
import numpy as np
import matplotlib.pyplot as plt




import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import os

from torchcontrib.optim import SWA

from apoNN.src.datasets import ApogeeDataset
from apoNN.src.utils import dump as dump 
from apoNN.src.utils import load as load 
from apoNN.src.vectors import *
from tagging.src.networks import ConditioningAutoencoder,Embedding_Decoder,Feedforward,ParallelDecoder,Autoencoder

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
apogee_path.change_dr(16)

In [7]:
n_data=6000

### Setup

In [22]:
allStar= apread.allStar(rmcommissioning=True,main=False,ak=True, akvers='targ',adddist=False)

upper_temp_cut = allStar["Teff"]<7000
lower_temp_cut = allStar["Teff"]>3500
lower_g_cut = allStar["logg"]>1.
upper_g_cut = allStar["logg"]<3.5
snr_cut = allStar["SNR"]>100
snr_highcut = allStar["SNR"]<500
"""feh_outliercut = allStar["Fe_H"]>-5
o_outliercut = allStar["O_FE"]>-5
c_outliercut = allStar["C_FE"]>-5
na_outliercut = allStar["Na_FE"]>-5
mg_outliercut = allStar["Mg_FE"]>-5
si_outliercut = allStar["Si_FE"]>-5
al_outliercut = allStar["Al_FE"]>-5
s_outliercut = allStar["S_FE"]>-5
p_outliercut = allStar["P_FE"]>-5
ti_outliercut = allStar["Ti_FE"]>-5
cr_outliercut = allStar["Cr_FE"]>-5"""



combined_cut = lower_g_cut & upper_g_cut & lower_temp_cut & upper_temp_cut & snr_cut & snr_highcut & feh_outliercut & o_outliercut &  c_outliercut & na_outliercut & mg_outliercut & si_outliercut & al_outliercut & p_outliercut & s_outliercut & ti_outliercut & cr_outliercut
cut_allStar = allStar[combined_cut]

In [27]:
pickled_dataset = ApogeeDataset(filename="aspcap_extended",outputs = ["aspcap","mask2","physical","idx"])
autoencoder = torch.load("/share/splinter/ddm/taggingProject/apogeeFactory/outputs/guild/z10/ae_3600.p")


In [12]:
z = LatentVector(pickled_dataset,autoencoder,n_data)

In [13]:
considered_parameters = ["Teff","logg","Fe_H","O_FE","C_FE","Na_FE","Mg_FE","Si_FE","S_FE","Al_FE","P_FE","Ni_FE"] 
y = Vector(np.array([cut_allStar[param] for param in considered_parameters])[:,:n_data].T)

### Linear algebra

With the linear algebra. Training a new model becomes incredibly easy.

In [ ]:
w = LinearTransformation(z,y)

In [ ]:
y_pred = w.predict(z)

In [ ]:
i = 9
plt.scatter(y.raw[:,i],y_pred.raw[:,i],s=0.5)
plt.title(considered_parameters[i])

In [ ]:
plt.scatter(y.raw[:,0],y.raw[:,1],s=0.5)


It's also possible to view the outputs for any directions. Not just those picked by the linear transformation matrix

In [ ]:
plt.scatter(y.raw[:,0],np.dot(w.val[0],z.centered.T),s=0.5)
#plt.scatter(y.raw[:,2],np.dot(test_pca,z.centered.T),s=0.6)

## Check the PCA breakdown

We can fit PCA to the latent space to find the directions of the hyperplane containing the data.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=10,whiten=True)

In [ ]:
pca.fit(z.centered)

In [ ]:
pca.explained_variance_ratio_

we see that not every single dimension contains as much variations. One trick is then we can transform the dataspace so as to have each direction contain equal variance

In [ ]:
z_whithened = pca.fit(pca.transform(z.centered))

In [ ]:
pca.explained_variance_ratio_

In [ ]:
import apoNN.src.utils as utils
utils.dump(pca,"pca")

In [ ]:
utils.load("pca")

In [ ]:
pca.components_[0]

In [ ]:
idx=8
np.dot(w.val[0:1]/np.linalg.norm(w.val[0:1]),w.val[idx:idx+1].T/np.linalg.norm(w.val[idx:idx+1]))

In [ ]:
w.val[0:1]/np.linalg.norm(w.val[0:1])

In [ ]:
test_pca = np.array([ -0.43056273, -0.6169323 ,  0.46758795,  0.42295858, -0.2486121 ,
        0.39564928, -0.0904031 ,  0.65999484,  0.91201407,  0.8673172 ,
       -0.13539782,  0.54779094,  0.8583661 ,  0.14308473,  0.0373497 ,
       -0.48964548, -0.0763603 , -0.23988001, -1.6411978 , -0.6619012 ,
       -0.6396435 , -0.44550008,  0.39345187, -0.6311279 ,  0.97478664])
idx=9
np.dot(test_pca,w.val[idx:idx+1].T/np.linalg.norm(w.val[idx:idx+1]))

In [ ]:
np.dot(w.val[0:1]/np.linalg.norm(w.val[0:1]),w.val[4:5].T/np.linalg.norm(w.val[4:5]))

In [ ]:
np.dot(w.val[0:1],w.val[3:4].T)

### Measure reconstruction

As a safeguard we measure the reconstruction of those stars within our dataset

In [ ]:
masked_loss

In [ ]:
loss = torch.nn.L1Loss()
for idx in range(100):
    x = pickled_dataset[idx][0].to(device).unsqueeze(0)
    x_pred,z = autoencoder(x)
    print(loss(x_pred,x))

In [ ]:
plt.plot(x_pred.detach().cpu().numpy()[0])
plt.plot(pickled_dataset[idx][0])
plt.xlim(5000,5500)
